In [1]:
import pandas as pd
import numpy as np
import math
from random import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from tqdm import tqdm
import matplotlib.animation as ani
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.stattools import adfuller as ADF
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

In [2]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
plt.rcParams["font.family"] = "Times New Roman"
plt.style.use('seaborn-whitegrid')
plt.style.use('seaborn-poster')
plt.style.use('seaborn-dark-palette')
plt.rcParams["mathtext.fontset"] = "cm"

In [3]:
# print all the columns
#pd.set_option('display.max_columns', None)
# print all the lines
#pd.set_option('display.max_rows', None)

# Forecasting (ARMA)

In [4]:
def adf_test(ts, signif=0.05):
    times = -1
    p = 1
    while (p > signif):
        print(times)
        times = times + 1
        dftest = ADF(ts)
        p = dftest[1]
        ts = ts.diff().dropna()
    return times
        
def order(train, times):
    tmp = []
    for p in range(1, 6):
        for q in range(5):
            try:
                tmp.append([ARIMA(train, order=(p, times, q)).fit().bic, p, q])
            except:
                tmp.append([None, p, q])
    tmp = pd.DataFrame(tmp,columns = ['bic', 'p', 'q'])
    return tmp[tmp['bic'] == tmp['bic'].min()]
    
def plot_time_series(ts_1, ts_label_1, ts_2, ts_label_2, title, path):
    assert len(ts_1) == len(ts_2)
    xs = list(range(0, len(ts_1)))

    plt.rcParams['savefig.dpi'] = 300 
    plt.rcParams['figure.dpi'] = 300
    
    plt.plot(xs, ts_2, c='red', label=ts_label_2, lw = 1)
    plt.plot(xs, ts_1, c='green', label=ts_label_1, lw = 1)

    plt.title(title)
    plt.legend(loc='upper left')
    plt.savefig(path)
    plt.show()
    
def comparaison_plot(ts_1, ts_label_1, ts_2, ts_label_2, ts_3, ts_label_3, title):
    assert len(ts_1) == len(ts_2)
    assert len(ts_2) == len(ts_3)
    
    xs = list(range(0, len(ts_1)))
    
    plt.rcParams['savefig.dpi'] = 300 
    plt.rcParams['figure.dpi'] = 300
    
    plt.plot(xs, ts_2, c='red', label=ts_label_2, lw = 1)
    plt.plot(xs, ts_3, c='blue', label=ts_label_3, lw = 1)
    plt.plot(xs, ts_1, c='green', label=ts_label_1, lw = 1)

    plt.title(title)
    plt.legend(loc='upper left')
    #plt.savefig(path)
    plt.show()
    
def ARIMA_model_loop(data, coef, times, radar): 
    
    train = data[:int(len(data)*coef)]
    test = data[int(len(data)*coef):]
    par = order(data, times)
    # Forecast
    start_t = len(train)
    predictions = list()
    for t in range(len(test)):        
        current_t = t + start_t
        model = ARIMA(data[:current_t], order=(par['p'], times, par['q']))       
        model_fit = model.fit()  
        predictions.append(model_fit.forecast().iloc[0])
        
    predictions = pd.DataFrame(predictions, columns = [radar])
    
    return predictions 

def ARIMA_model(data, coef, times):
    train = data[:int(len(data)*coef)]
    test = data[int(len(data)*coef):]
    par = order(data, times)
    # Forecast
    model = ARIMA(train, order=(par['p'], times, par['q']))       
    model_fit = model.fit()  
    predictions = pd.DataFrame(model_fit.forecast(len(test)), columns = ['radar'])
    
    return predictions

## ARIMA with loop

In [ ]:
df_forecasting = pd.read_csv('../dataset/time_filled_AR.csv')
#df_forecasting = df_forcasting.drop(['samplingperiod'], axis = 1)
coef = 0.5
#radar = 'KABX' 
pred = pd.DataFrame()
for radar in tqdm(df_forecasting.columns.drop('samplingperiod')):

    times = adf_test(df_forecasting[radar])
    pred[radar] = ARIMA_model_loop(df_forecasting[radar], coef, times, radar)
    print(pred)
    #plot_time_series(ts_1=pred[radar], ts_label_1='ARIMA Model', ts_2=df_forecasting[radar], ts_label_2='True data', title='ARIMA predictions vs. ground truth of %s'%radar, path = '../figures/bird/%s.png'%radar)

pred.to_csv('../dataset/pred_ARIMA.csv', index = False)

## ARIMA with log

In [ ]:
data = pd.read_csv('../dataset/time_filled_AR.csv')
#print(data.isnull().sum())
#df_forecasting = df_forcasting.drop(['samplingperiod'], axis = 1)

df_forecasting = np.log10(data[data.columns.drop('samplingperiod')]+1)
coef = 0.5
pred = pd.DataFrame()
#radar = 'KAPX' 
#print(df_forecasting[radar])
for radar in tqdm(df_forecasting.columns):
    
    times = adf_test(df_forecasting[radar])
    pred[radar] = np.power(10, ARIMA_model_loop(df_forecasting[radar], coef, times, radar))-1
    #plot_time_series(ts_1=pred[radar], ts_label_1='ARIMA Model', ts_2=data[radar], ts_label_2='True data', title='ARIMA predictions vs. ground truth of %s'%radar, path = '../figures/bird/%s.png'%radar)
    print(pred)
    
pred.to_csv('../dataset/pred_log.csv', index = False)


## ARIMA without loop

In [ ]:
df_forecasting = pd.read_csv('../dataset/time_filled_AR.csv')
coef = 0.5
pred = pd.DataFrame()
#radar = 'KABX'
for radar in tqdm(df_forecasting.columns.drop('samplingperiod')):
    
    times = adf_test(df_forecasting[radar])
    pred[radar] = ARIMA_model(df_forecasting[radar], coef, times)
    #plot_time_series(ts_1=pred[radar], ts_label_1='ARIMA Model', ts_2=time.radar[radar], ts_label_2='True data', title='ARIMA predictions vs. ground truth of %s'%radar, path = '../figures/bird/%s.png'%radar)
    print(pred)
    
pred.to_csv('../dataset/pred_ARIMA_noloop.csv', index = False)


## Comparison between three methods

In [ ]:
comparaison_plot(ts_1 = pred[int(len(pred)*coef)-2:], ts_label_1 = 'ARIMA loop', ts_2 = time.radar[radar][int(len(pred)*coef)-2:], 
                 ts_label_2 = 'true data', ts_3 = pred1[int(len(pred)*coef)-2:], ts_label_3 = 'ARIMA', 
                 title = 'comparison of ARIMA and ARIMA loop')
comparaison_plot(ts_1 = pred, ts_label_1 = 'ARIMA loop', ts_2 = time.radar[radar], 
                 ts_label_2 = 'true data', ts_3 = pred1, ts_label_3 = 'ARIMA', 
                 title = 'comparison of ARIMA and ARIMA loop')

# Forecasting (VAR)

In [ ]:
time_radar = pd.read_csv('../dataset/time_filled_ARIMA.csv')
coef = 0.7
#apply adf test on the series
test = time_radar[int(coef*(len(time.radar))):]
train = time_radar[:int(coef*(len(time.radar)))]
predictions = train
#start_t = len(train)
#for t_i in tqdm(range(len(test))):
#current_t = t_i + start_t
model = VAR(train)#time.radar[:current_t])
model_fit = model.fit()
prediction = model_fit.forecast(model_fit.y, steps=len(test))
prediction = pd.DataFrame(prediction, columns = time_radar.columns)
predictions = pd.concat([predictions, prediction], axis = 0)
predictions.reset_index(drop = True, inplace = True)

In [ ]:
for radar in predictions.columns:
    plot_time_series(ts_1=predictions[radar], ts_label_1='VAR prediction', ts_2=time.radar[radar], ts_label_2='True data', title='VAR predictions vs. ground truth of %s'%radar, path = '../figures/bird/VAR/%s.png'%radar)


In [ ]:
predictions.to_csv('../dataset/pred_var.csv', index = False)

# Forecasting(ARMAX)

In [ ]:
def order_armax(train, data_ex, times):
    tmp = []
    for p in tqdm(range(1, 6)):
        for q in tqdm(range(5)):
            try:
                tmp.append([ARIMA(train, exog = data_ex, order=(p, times, q)).fit().bic, p, q])
            except:
                tmp.append([None, p, q])
    tmp = pd.DataFrame(tmp,columns = ['bic', 'p', 'q'])
    return tmp[tmp['bic'] == tmp['bic'].min()]

def ARMAX_model(data, data_ex, coef, times):

    train = data[:int(len(data)*coef)]
    test = data[int(len(data)*coef):]
    par = order_armax(train, data_ex[:int(len(data)*coef)], times)
    # Forecast
    start_t = len(train)
    predictions = list()
    for t in range(len(test)): 
        
        current_t = t + start_t
        model = ARIMA(data[:current_t], exog = data_ex[:current_t], order=(par['p'], times, par['q']))       
        model_fit = model.fit()  
        
        print(model_fit.predict())
        #print(model_fit.summary())
        model_fit.plot_diagnostics(figsize=(12, 12))

        #print(model_fit.forecast())
        predictions.append(model_fit.predict())
        
    predictions = pd.DataFrame(predictions)
    predictions = pd.concat([train, predictions], axis = 0)
    predictions.reset_index(inplace = True, drop = True)
    
    return predictions 



In [ ]:
time_radar = pd.read_csv('../dataset/radar/time_filled_ARIMA.csv')
data_w = pd.read_csv('../dataset/radar/weather_information.csv')
radars = time_radar.columns
coef = 0.5
times = 0
weather1=['uwind','vwind','air','pressure.sfc','relative.humidity','ordinal.date', 
         'omega', 'total.cloud.cover', 'visibility', 'albedo', 'acc.total.precip', 'msl.pressure','cape','snow.cover']
#for radar in radars:
radar = 'KABX'
print(len(time_radar[radar]))
print(len(data_w[data_w['radar_id'] == radar][weather1]))
pred = ARMAX_model(time_radar[radar], data_w[data_w['radar_id'] == radar][weather1], coef, times)


In [ ]:
plot_time_series(ts_1=pred, ts_label_1='ARIMA Model', ts_2=data.density[data.density['radar_id'] == radar]['linear_eta'], ts_label_2='True data', title='ARIMA predictions vs. ground truth of %s'%radar, path = '../figures/bird/%s.png'%radar)
